In [1]:
## load modules and run mlflow_logging.ipynb to get function to track model information on MLFLow
import sys
sys.path.append("..")
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.svm import LinearSVR, SVR
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler


/Users/fklein/neuefische/capstone/Capstone_WindPowerPredicting/.venv/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
class Yint(int):
    def __new__(cls, value, payload):
        x = int.__new__(cls, value)
        x.payload = payload
        return x
 
# This applies for float and str, but not bool
 
# bool is derived from int, but can't be subclassed itself
class Ybool(int):
    def __new__(cls, value, payload):
        x = int.__new__(cls, bool(value))
        x.payload = payload
        return x

In [34]:
class positive(int):
    def __new__(cls, value, *args, **kwargs):
        # if value < 0:
        #     raise ValueError("positive types must not be less than zero")
        return  super(cls, cls).__new__(cls, value)

    def __add__(self, other):
        res = super(positive, self).__add__(other)
        return self.__class__(max(res, 0))

    def __sub__(self, other):
        res = super(positive, self).__sub__(other)
        res = (res+360)%360
        return self.__class__(max(res, 0))

    def __mul__(self, other):
        res = super(positive, self).__mul__(other)
        return self.__class__(max(res, 0))

    def __div__(self, other):
        res = super(positive, self).__div__(other)
        return self.__class__(max(res, 0))

    def __str__(self):
        return "%d" % int(self)

    def __repr__(self):
        return "positive(%d)" % int(self)



In [35]:
a = positive(3)
b = positive(-5)

In [36]:
b

positive(-5)

In [38]:
a-b

positive(8)